# Impact of RNN Architecture - Code Part I

In [108]:
import numpy as np
import statistics as st
import math
import tensorflow as tf
from keras.layers import GRU, SimpleRNN, Embedding, Dense, LSTM, Dropout
from keras.models import Sequential
from keras.metrics import Precision, Recall
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
import nltk

### Data pre-processing

In [219]:
# Loading the dataset

full_dataset = pd.read_csv("C:/Users/natal/CSCI5922_NN/Lab3/spam_detection_rnn/data/SPAM text message 20170820 - Data.csv")

In [220]:
full_text = []
labels = []

for x, label in enumerate(full_dataset['Category']):
    full_text.append(full_dataset['Message'][x])
    if label == 'ham':
        labels.append(0)
    else:
        labels.append(1)

full_text = np.asarray(full_text)
labels = np.asarray(labels)

In [221]:
print("Size of separate messages: ", len(full_text))
print("Size of labels: ", len(labels))

Size of separate messages:  5572
Size of labels:  5572


### Dataset split - for basic full test split (all sequence lengths)

In [222]:
# features - number of words
max_features = 10000

# Splitting the data
train = int(5572 * .7)
#print(train)
test = int(5572 - train)
#print(test)

# Checking the shapes are correct
print(len(full_text) == (train + test))
print("Train size: ", train, "\nTest size: ", test)

# Tokenizning 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(full_text)
sequences = tokenizer.texts_to_sequences(full_text)

# after, stop putting more words
max_len = 500

idx_word = tokenizer.word_index
print("Tokens size: ", len(idx_word))

padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Checking the shape
print("Data size after tokenizing: ", padded_sequences.shape)

np.random.seed(42)
indices = np.arange(padded_sequences.shape[0])
np.random.shuffle(indices)
padded_sequences_idx = padded_sequences[indices]
labels = labels[indices]

x_train = padded_sequences_idx[:train]
y_train = labels[:train]
x_test = padded_sequences_idx[train:]
y_test = labels[train:]

# Checking the shapes after splitting
print("Train x shape after split: ", x_train.shape, "\nTrain y shape after split: ", y_train.shape)
print("Test x shape after split: ", x_test.shape, "\nTest y shape after split: ", y_test.shape)

True
Train size:  3900 
Test size:  1672
Tokens size:  9004
Data size after tokenizing:  (5572, 500)
Train x shape after split:  (3900, 500) 
Train y shape after split:  (3900,)
Test x shape after split:  (1672, 500) 
Test y shape after split:  (1672,)


### Analysis of the messages' length - necessary for further splitting the test set 

In [223]:
# Checking the length of messages to perform the division of the test set based on the number of words
# Creating the boundaries for: Short, medium and long inputs

texts_list = full_text.tolist()
print("\n------------------------------------------------------\n")
print(f"Sentence example from the messages:\n\n {texts_list[5]}")
print("\n------------------------------------------------------\n")

def tokenizing():
    splitted = []
    for x in texts_list:
        splitted.append(x.split(' '))
    return splitted

splitted_messages = tokenizing()
print(f"Checking the number of messages: {len(splitted_messages)}")
print("\n------------------------------------------------------\n")
print(f"Sentence example after tokenizing: \n\n {splitted_messages[5]}")
print("\n------------------------------------------------------\n")

def counting_length():
    lengths = []
    for x in splitted_messages:
        lengths.append(len(x))
    return lengths

print(f"Length of the first 20 messages: \n\n{counting_length()[:20]}")
print("\n------------------------------------------------------\n")

sorted_counts = sorted(counting_length(), reverse=True)
print(f"Sorted list; descending:\n\n{sorted_counts[:20]}")

div = math.floor(len(texts_list)/3)
long_lenghts = sorted_counts[:div]
print("\n------------------------------------------------------\n")
print(f"The mean of the first set in the desceding list (large): {st.mean(long_lenghts)}")

medium_lenghts = sorted_counts[div:int(2*div)]
print("\n------------------------------------------------------\n")
print(f"The mean of the first set in the desceding list (medium): {st.mean(medium_lenghts)}")
print(f"Lengths of 20 messages before the frist division: {sorted_counts[div-20:int(2*div)][:20]}")

short_lenghts = sorted_counts[2*div:]
print("\n------------------------------------------------------\n")
print(f"The mean of the first set in the desceding list (short): {st.mean(short_lenghts)}")
print(f"Lengths of 20 messages before the second division: {sorted_counts[(2*div-20):][:20]}")
print("\n------------------------------------------------------\n")

short=9
medium=18
long=170
print(f"Chosen boundaries for lengths:\n\nShort: {short}\nMedium: {medium}\nLong: {long}")


------------------------------------------------------

Sentence example from the messages:

 FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv

------------------------------------------------------

Checking the number of messages: 5572

------------------------------------------------------

Sentence example after tokenizing: 

 ['FreeMsg', 'Hey', 'there', 'darling', "it's", 'been', '3', "week's", 'now', 'and', 'no', 'word', 'back!', "I'd", 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still?', 'Tb', 'ok!', 'XxX', 'std', 'chgs', 'to', 'send,', '£1.50', 'to', 'rcv']

------------------------------------------------------

Length of the first 20 messages: 

[20, 6, 28, 11, 13, 32, 16, 26, 26, 29, 21, 26, 26, 37, 8, 19, 4, 19, 13, 24]

------------------------------------------------------

Sorted list; descending:

[171, 162, 125, 125, 121, 119, 99, 96, 96, 95, 89, 88, 80, 79, 79, 79

### Test data preparation for test-size experiments based on the sequence length

In [233]:
# Creating a dataframe with messages and word counts

#Checking the lengths of texts and labels
len(texts_list)
len(labels)

#Making sure the numpy arrays are converted to lists
type(texts_list)
type(labels)

labels_list = labels.tolist()

type(labels_list)

#Creating a dictionary and then making a dataframe 
dict_full_dataset = {'messages': texts_list, 'labels': labels_list}
df_full_dataset = pd.DataFrame(dict_full_dataset)
df_full_dataset.head()

#Creating an additional column with word counts
df_full_dataset['word_count'] = df_full_dataset['messages'].str.split().str.len()
df_full_dataset

#Setting df options to display example messages
pd.set_option('display.max_colwidth', None)

#Creating three new dataframes based on the conditions of the length of seqeunces
#Boundaries for nummber of words were created in the previous cells with the analysis of the dataset
df_short_messages = df_full_dataset[df_full_dataset['word_count']<short]
print("\n------------------------------------------------------\n")
print(f"Length of short messages dataset: {len(df_short_messages)}\n")
print(f"Example of a message: {df_short_messages['messages'].sample(1,ignore_index=True)}")

df_medium_messages = df_full_dataset[(df_full_dataset['word_count']<=medium) & (df_full_dataset['word_count']>=short)]
print("\n------------------------------------------------------\n")
print(f"Length of medium messages dataset: {len(df_medium_messages)}\n")
print(f"Example of a message: {df_medium_messages['messages'].sample(1,ignore_index=True)}")

df_long_messages = df_full_dataset[df_full_dataset['word_count']>medium]
print("\n------------------------------------------------------\n")
print(f"Length of long messages dataset: {len(df_long_messages)}\n")
print(f"Example of a long message: {df_long_messages['messages'].sample(1,ignore_index=True)}")

#Reset df options
pd.reset_option('display.max_colwidth')


------------------------------------------------------

Length of short messages dataset: 1861

Example of a message: 0    Yeah I am, so I'll leave maybe 7ish?
Name: messages, dtype: object

------------------------------------------------------

Length of medium messages dataset: 1843

Example of a message: 0    Did you get any gift? This year i didnt get anything. So bad
Name: messages, dtype: object

------------------------------------------------------

Length of long messages dataset: 1868

Example of a long message: 0    Collect your VALENTINE'S weekend to PARIS inc Flight & Hotel + £200 Prize guaranteed! Text: PARIS to No: 69101. www.rtf.sphosting.com
Name: messages, dtype: object


### Splitting the dataset accroding to sequence size for experiments

In [241]:
#Extracting lists from df of short messages and checking length are compatible
short_list=df_short_messages['messages'].to_list()
short_list_labels=df_short_messages['labels'].to_list()
print(f"Checking the length of messages == length of labels: {len(short_list)==len(short_list_labels)}")

#Extracting lists from df of medium messages and checking length are compatible
m_list=df_medium_messages['messages'].to_list()
m_list_labels=df_medium_messages['labels'].to_list()
print(f"Checking the length of messages == length of labels: {len(m_list)==len(m_list_labels)}")

#Extracting lists from df of long messages and checking length are compatible
l_list=df_long_messages['messages'].to_list()
l_list_labels=df_long_messages['labels'].to_list()
print(f"Checking the length of messages == length of labels: {len(l_list)==len(l_list_labels)}")

Checking the length of messages == length of labels: True
Checking the length of messages == length of labels: True
Checking the length of messages == length of labels: True


In [245]:
#Creating numpy arrays
full_text_short = np.asarray(short_list)
labels_short = np.asarray(short_list_labels)

full_text_medium = np.asarray(m_list)
labels_medium = np.asarray(m_list_labels)

full_text_long = np.asarray(l_list)
labels_long = np.asarray(l_list_labels)
print(type(labels_long))

<class 'numpy.ndarray'>


In [ ]:
#Since the full test data has ~1600 entries, I will not further split the created sets into train/test
#The current sets are ~1800 so it's comparable to the original full test set

#Steps as above to vectorize the sequences of words

def vectorize_data(x,y):
    # Tokenizning
    tokenizer = Tokenizer()
    fitted = tokenizer.fit_on_texts(x)
    sequences = tokenizer.texts_to_sequences(x)

    # after, stop putting more words
    max_len = 500

    idx_word = tokenizer.word_index
    print("Tokens size: ", len(idx_word))

    padded = pad_sequences(sequences, maxlen=max_len)

    # Checking the shape
    print("Data size after tokenizing: ", padded.shape)

    np.random.seed(42)
    indices = np.arange(padded.shape[0])
    np.random.shuffle(indices)
    padded_sequences_idx = padded[indices]
    labels = y[indices]

    x_data = padded_sequences_idx[:]
    y_data = labels[:]

    return x_data, y_data

In [248]:
x_test_short, y_test_short = vectorize_data(full_text_short, labels_short)
x_test_medium, y_test_medium = vectorize_data(full_text_medium, labels_medium)
x_test_long, y_test_long = vectorize_data(full_text_long, labels_long)

Tokens size:  2357
Data size after tokenizing:  (1861, 500)
Tokens size:  3846
Data size after tokenizing:  (1843, 500)
Tokens size:  6659
Data size after tokenizing:  (1868, 500)


## Models

### Test Model

In [25]:
# Vanilla test RNN model

model = Sequential()
model.add(Embedding(max_features, 64))
model.add(SimpleRNN(64,input_shape=x_train.shape,return_sequences=False, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer ='rmsprop',metrics=['acc'])

model_rnn = model.fit(x_train, y_train, epochs = 15, batch_size=32)

Epoch 1/15
122/122 [==============================] - 14s 101ms/step - loss: 0.2278 - acc: 0.9208
Epoch 2/15
122/122 [==============================] - 13s 106ms/step - loss: 0.0577 - acc: 0.9831
Epoch 3/15
122/122 [==============================] - 13s 104ms/step - loss: 0.0397 - acc: 0.9892
Epoch 4/15
122/122 [==============================] - 13s 105ms/step - loss: 0.0271 - acc: 0.9923
Epoch 5/15
122/122 [==============================] - 13s 105ms/step - loss: 0.0136 - acc: 0.9956
Epoch 6/15
122/122 [==============================] - 13s 107ms/step - loss: 0.0097 - acc: 0.9969
Epoch 7/15
122/122 [==============================] - 13s 109ms/step - loss: 0.0065 - acc: 0.9987
Epoch 8/15
122/122 [==============================] - 13s 108ms/step - loss: 0.0033 - acc: 0.9992
Epoch 9/15
122/122 [==============================] - 13s 106ms/step - loss: 0.0019 - acc: 0.9995
Epoch 10/15
122/122 [==============================] - 13s 104ms/step - loss: 0.0017 - acc: 0.9995
Epoch 11/15
122/122

### Analysis of first model_rnn

After running the very first model, it was observed that the model is probably overfitting the data. The accuracy after the epoch 15 is 100% and the model probably learns the noise. The model is not learning objectively and is fit too much to the training data. Before I actually start runnig the 3 models (vanilla RNN, LSTM and GRU), I will treat the model overfitting:
* reducing the network's capacity --> decreasing the number of units in the model's layers
* applying regularization technique --> penalizing very large weights
* changing optimizer to Adam
* increasing the batch size --> the model will learn lesser noise; it will help take a more reasonable 'step' for minima
* adding dropout layers --> by dropping some layer of the network, I will let the model generalize better on unseen examples; it won't be trained for details and noises of the training data

I will call this model a test model, apply the mentioned changes and start building the 3 RNN models for the experiments.

In [40]:
# Vanilla RNN model 1 

model = Sequential()
model.add(Embedding(max_features, 15))
model.add(SimpleRNN(15,input_shape=x_train.shape, activation='tanh',recurrent_regularizer='l2'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics=['acc', Precision(), Recall()])

model_vanilla_rnn = model.fit(x_train, y_train, epochs = 15, batch_size=256)
print("---------------------------------------------------")
print("Model evaluation on the full test set:")
model.evaluate(x_test,y_test, verbose=1, return_dict=True)

Epoch 1/15
16/16 [==============================] - 7s 161ms/step - loss: 0.8031 - acc: 0.6656 - precision_3: 0.1346 - recall_3: 0.2759
Epoch 2/15
16/16 [==============================] - 3s 168ms/step - loss: 0.6366 - acc: 0.8669 - precision_3: 1.0000 - recall_3: 0.0057
Epoch 3/15
16/16 [==============================] - 3s 160ms/step - loss: 0.5296 - acc: 0.8662 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00
Epoch 4/15
16/16 [==============================] - 2s 154ms/step - loss: 0.4541 - acc: 0.8828 - precision_3: 1.0000 - recall_3: 0.1245
Epoch 5/15
16/16 [==============================] - 2s 154ms/step - loss: 0.3370 - acc: 0.9741 - precision_3: 0.9976 - recall_3: 0.8084
Epoch 6/15
16/16 [==============================] - 2s 154ms/step - loss: 0.2595 - acc: 0.9856 - precision_3: 0.9936 - recall_3: 0.8985
Epoch 7/15
16/16 [==============================] - 3s 172ms/step - loss: 0.2240 - acc: 0.9887 - precision_3: 0.9938 - recall_3: 0.9215
Epoch 8/15
16/16 [======================

{'loss': 0.23030498623847961,
 'acc': 0.9766746163368225,
 'precision_3': 0.9946808218955994,
 'recall_3': 0.8311111330986023}

In [30]:
# Vanilla LSTM model

model = Sequential()
model.add(Embedding(max_features, 15))
model.add(LSTM(15,input_shape=x_train.shape, activation='tanh',recurrent_regularizer='l2'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics=['acc'])

model_lstm = model.fit(x_train, y_train, epochs = 10, batch_size=256)
model.evaluate(x_test,y_test, verbose=1, return_dict=True)

Epoch 1/10
16/16 [==============================] - 9s 310ms/step - loss: 0.8018 - acc: 0.8449
Epoch 2/10
16/16 [==============================] - 5s 282ms/step - loss: 0.6826 - acc: 0.8662
Epoch 3/10
16/16 [==============================] - 4s 276ms/step - loss: 0.4927 - acc: 0.8662
Epoch 4/10
16/16 [==============================] - 4s 276ms/step - loss: 0.3997 - acc: 0.8662
Epoch 5/10
16/16 [==============================] - 5s 318ms/step - loss: 0.3187 - acc: 0.8900
Epoch 6/10
16/16 [==============================] - 5s 333ms/step - loss: 0.2515 - acc: 0.9341
Epoch 7/10
16/16 [==============================] - 5s 319ms/step - loss: 0.2042 - acc: 0.9667
Epoch 8/10
16/16 [==============================] - 5s 337ms/step - loss: 0.1733 - acc: 0.9790
Epoch 9/10
16/16 [==============================] - 5s 309ms/step - loss: 0.1504 - acc: 0.9838
Epoch 10/10
16/16 [==============================] - 5s 307ms/step - loss: 0.1316 - acc: 0.9882


In [31]:
# Vanilla LSTM model

model = Sequential()
model.add(Embedding(max_features, 15))
model.add(GRU(15,input_shape=x_train.shape, activation='tanh',recurrent_regularizer='l2'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics=['acc'])

model_gru = model.fit(x_train, y_train, epochs = 10, batch_size=256)
model.evaluate(x_test,y_test, verbose=1, return_dict=True)

Epoch 1/10
16/16 [==============================] - 10s 281ms/step - loss: 0.8059 - acc: 0.8336
Epoch 2/10
16/16 [==============================] - 4s 280ms/step - loss: 0.7132 - acc: 0.8662
Epoch 3/10
16/16 [==============================] - 4s 278ms/step - loss: 0.5864 - acc: 0.8662
Epoch 4/10
16/16 [==============================] - 5s 285ms/step - loss: 0.4391 - acc: 0.8662
Epoch 5/10
16/16 [==============================] - 5s 288ms/step - loss: 0.3587 - acc: 0.8674
Epoch 6/10
16/16 [==============================] - 5s 286ms/step - loss: 0.2896 - acc: 0.9082
Epoch 7/10
16/16 [==============================] - 5s 282ms/step - loss: 0.2279 - acc: 0.9503
Epoch 8/10
16/16 [==============================] - 5s 283ms/step - loss: 0.1799 - acc: 0.9687
Epoch 9/10
16/16 [==============================] - 5s 284ms/step - loss: 0.1464 - acc: 0.9821
Epoch 10/10
16/16 [==============================] - 5s 288ms/step - loss: 0.1230 - acc: 0.9874
